# Image Thresholding
If value is above, it is object
If it is below or equal, it is background


In [2]:
#Load images
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("Hardware.png", cv2.IMREAD_GRAYSCALE)
imgNu = cv2.imread("Nuclei.png", cv2.IMREAD_GRAYSCALE)
imgOr = cv2.imread("Orca.png", cv2.IMREAD_GRAYSCALE)


In [7]:
# Hardware image    

# Otsu's Method
ret1,th1 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

#IsoData method
t = 0.5
while True:
    # compute mean of first class
    mu1 = img[img<=t].mean()
    mu2 = img[img>t].mean()
    tNew = (mu1 + mu2)/2
    if (abs(t - tNew) < 0.01):
       break
    t = tNew
    print("Isodata Threshold = "+ t.astype(str))


ret2,th2 = cv2.threshold(img,255*t,255,cv2.THRESH_BINARY)

#Triangle method
imgHist = cv2.calcHist(img)




# Show images
images = [img, 0, th1, th2, th3]
titles = ['Original Image','Histogram',"Otsu's Thresholding","IsoData Method","Triangle Method"]

f, axarr  = plt.subplot(3,3)
axarr[0].imshow(img)
axarr[1].imshow(img)
axarr[2].imshow(img)
axarr[3].imshow(img)

plt.show()


C:\Users\shady\AppData\Local\Temp\ipykernel_23848\2530335688.py:10: RuntimeWarning: Mean of empty slice.
  mu1 = imgHw[imgHw<=t].mean()
C:\Users\shady\AppData\Local\Temp\ipykernel_23848\2530335688.py:11: RuntimeWarning: Mean of empty slice.
  mu2 = imgHw[imgHw>t].mean()


Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshold = nan
Isodata Threshol

KeyboardInterrupt: 

In [ ]:
# Nuclei

In [ ]:
#Orca